In [6]:
# set the environment path to find Recommenders
import sys

import itertools
import logging
import os

import numpy as np
import pandas as pd
import papermill as pm

from recommendersmain import recommenders as r


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]
Pandas version: 1.5.3
